In [12]:
# import cv2

# video_path = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV"

# cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Video", height=760, width=1366) 

# scanned_video = cv2.VideoCapture(video_path)
# frame_count = 1

# if (scanned_video.isOpened()== False): 
#     print("Error opening video stream or file")

# while (scanned_video.isOpened()):

#     ret, frame = scanned_video.read()

#     if ret == True:
#         cv2.imshow("Video", frame)
#         # frame_count = frame_count + 1
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#         if(cv2.waitKey(1) & 0xFF == ord('p')):
#             cv2.waitKey(-1)
#     else:
#         break

# scanned_video.release()
# cv2.destroyAllWindows()

In [13]:
import cv2


video_path = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV"

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', height=760, width=1366) 

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255,255,255)
lineType = 2

cap = cv2.VideoCapture(video_path)
frame_count = 1

while True:

    ret, frame = cap.read()
    key = cv2.waitKey(1) & 0xff

    if not ret:
        break

    if key == ord('p'):

        while True:

            key2 = cv2.waitKey(1) or 0xff
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow('frame', frame)

            if key2 == ord('p'):
                print(frame_count)
                break

    cv2.imshow('frame',frame)
    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
    frame_count = frame_count + 1

    if key == 27: 
        break

cap.release()
cv2.destroyAllWindows()

97
121
140
155
170
186
